In [1]:
import torch
import sys
sys.path.append('/home/zjy/project/MetaIM')
pwd = '/home/zjy/project/MetaIM/data'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=1)

In [2]:
from torch_geometric.datasets import Planetoid

cora_dataset = Planetoid(root=pwd+'/cora', name='cora')
data = cora_dataset[0]
edge_index = data.edge_index

In [3]:
import numpy as np
individual_infection_path = pwd+'/for_meta/cora_individual_infection_sir_200.npy'
seeds_infection_path = pwd+'/for_meta/cora_seed_infection_sir_200_sample_1000.npy'

individual_infection = np.load(individual_infection_path)
seeds_infection = np.load(seeds_infection_path)
individual_infection.shape,seeds_infection.shape

((2708, 2708), (1000, 2, 2708))

In [4]:
import torch
from torch_geometric.utils import to_scipy_sparse_matrix
import scipy.sparse as sp

# 转换为 scipy 稀疏矩阵
adj = to_scipy_sparse_matrix(edge_index)

adj = torch.Tensor(adj.toarray()).to_sparse()
adj = adj.to(device)


In [5]:
seed_num = int(seeds_infection[0][0].sum())

In [6]:
from torch.utils.data import Dataset, DataLoader, random_split


class CustomDataset(Dataset):
    def __init__(self, individual_infection,seeds_infection):
        self.individual_infection = individual_infection
        self.seeds_infection = seeds_infection

    def __len__(self):
        return len(self.seeds_infection)

    def __getitem__(self, idx):
        
        return self.seeds_infection[idx][0], self.seeds_infection[idx][1]

dataset = CustomDataset(individual_infection, seeds_infection)

In [7]:
# 定义划分比例
train_ratio = 0.8
test_ratio = 0.2

# 划分数据集
train_dataset, test_dataset = random_split(dataset, [int(len(dataset)*train_ratio), int(len(dataset)*test_ratio)])

train_batch_size = 32
test_batch_size = 2

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

训练VAE

In [8]:
# from data import model 
from data.model.model import VAEModel, Encoder, Decoder
from torch.optim import Adam, SGD
import torch.nn.functional as F

# # hidden_dim = 256
# # latent_dim = 64
hidden_dim = 1024
latent_dim = 128

encoder = Encoder(input_dim= len(seeds_infection[0][0]), 
                  hidden_dim=hidden_dim, 
                  latent_dim=latent_dim)

decoder = Decoder(input_dim=latent_dim, 
                  latent_dim=latent_dim, 
                  hidden_dim=hidden_dim, 
                  output_dim=len(seeds_infection[0][0]))

vae_model = VAEModel(Encoder=encoder, Decoder=decoder).to(device)

optimizer_vae = Adam([{'params': vae_model.parameters()}], 
                 lr=1e-3)
vae_model.train()

VAEModel(
  (Encoder): Encoder(
    (FC_input): Linear(in_features=2708, out_features=1024, bias=True)
    (FC_input2): Linear(in_features=1024, out_features=1024, bias=True)
    (FC_output): Linear(in_features=1024, out_features=128, bias=True)
    (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Decoder): Decoder(
    (FC_input): Linear(in_features=128, out_features=128, bias=True)
    (FC_hidden_1): Linear(in_features=128, out_features=1024, bias=True)
    (FC_hidden_2): Linear(in_features=1024, out_features=1024, bias=True)
    (FC_output): Linear(in_features=1024, out_features=2708, bias=True)
  )
)

In [9]:
for epoch in range(200):
    train_vae_loss = 0
    mean_train_accuracy = 0
    for batch_idx, seeds_label in enumerate(train_loader):        
        x = seeds_label[0].to(device)
        optimizer_vae.zero_grad()
        loss = 0
        for i, x_i in enumerate(x):
            x_hat = vae_model(x_i)

            # reproduction_loss = F.binary_cross_entropy(x_hat, x_i, reduction='sum')   
            reproduction_loss = F.mse_loss(x_hat, x_i, reduction='sum')   
            loss += reproduction_loss    
        train_vae_loss += loss.item()
        loss = loss/x.size(0)
        loss.backward()
        optimizer_vae.step()
        
    print("Epoch: {}".format(epoch+1), 
        "\tTrain_vae_loss: {:.4f}".format(train_vae_loss / train_batch_size),
        )


Epoch: 1 	Train_vae_loss: 5366.7296
Epoch: 2 	Train_vae_loss: 3375.0000
Epoch: 3 	Train_vae_loss: 3375.0000
Epoch: 4 	Train_vae_loss: 3375.0000
Epoch: 5 	Train_vae_loss: 3375.0000
Epoch: 6 	Train_vae_loss: 3375.0000
Epoch: 7 	Train_vae_loss: 3375.0000


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn

from torch_geometric.nn import GATConv
from torch.optim import Adam, SGD
import torch.nn.functional as F

class GAT(nn.Module):
    def __init__(self, seeds_dim, inflect_dim, hidden_channels, out_channels, num_heads):
        super(GAT, self).__init__()
        self.linear1 = nn.Linear(inflect_dim, 2 * inflect_dim)
        self.bn1 = nn.BatchNorm1d(2 * inflect_dim)
        self.conv1 = GATConv(2 * inflect_dim, hidden_channels, heads=num_heads)
        self.bn2 = nn.BatchNorm1d(hidden_channels * num_heads)
        self.conv2 = GATConv(hidden_channels * num_heads, hidden_channels * num_heads, heads=1)
        self.bn3 = nn.BatchNorm1d(seeds_dim)
        self.linear2 = nn.Linear(hidden_channels * num_heads + seeds_dim, out_channels)

    def forward(self, seeds_i, inflect_i, edge_index):
        x =  torch.cat((seeds_i, inflect_i), dim=-1)
        x = self.linear1(x)
        x = self.bn1(x)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.bn2(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        inflect_i = self.bn3(inflect_i)
        x =  torch.cat((x, inflect_i), dim=-1)
        x = self.linear2(x)
        return F.relu(x)

In [ ]:
inflect_dim = latent_dim
seeds_dim = latent_dim

forward_model = GAT(seeds_dim,inflect_dim, 512, 1, 4)

optimizer = Adam([{'params': forward_model.parameters()}], 
                 lr=0.0001)

adj = adj.to(device)
forward_model = forward_model.to(device)
forward_model.train()

GAT(
  (linear1): Linear(in_features=256, out_features=256, bias=True)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): GATConv(256, 512, heads=4)
  (bn2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): GATConv(2048, 2048, heads=1)
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=2176, out_features=1, bias=True)
)

In [ ]:
for param in vae_model.parameters():
    param.requires_grad = False 
encoder = vae_model.Encoder

In [ ]:
inflected = torch.tensor(individual_infection).T.to(device)
encode_inflected = torch.zeros(inflected.shape[0], latent_dim)
for i in range(inflected.shape[0]):
    encode_inflected_i  = encoder(inflected[i])
    encode_inflected[i] = encode_inflected_i
encode_inflected = encode_inflected.detach().to(device)
encode_inflected

tensor([[-0.9710, -0.7823, -0.2452,  ...,  0.5213, -0.9167, -0.8045],
        [-0.8909, -0.7296, -0.1646,  ...,  0.3429, -0.9508, -1.1218],
        [ 2.6712, -0.5249,  0.6462,  ...,  0.2255, -2.4219,  0.3254],
        ...,
        [-0.9943, -1.0166, -0.1454,  ...,  0.3835, -0.4427, -0.2174],
        [-0.2526, -0.8913,  0.0411,  ...,  0.2163, -0.9409, -0.1455],
        [ 0.8175, -1.1660,  0.1499,  ...,  0.2104, -1.7778,  0.3199]],
       device='cuda:1')

In [ ]:
edge_index = edge_index.to(device)
top_num = 500


for epoch in range(2000):

    total_loss = 0
    

    total_train_accuracy = 0
    
    count_train = 0
    for batch_idx, seeds_label in enumerate(train_loader): 
        count_train += 1
        forward_loss = 0 
        seeds =  seeds_label[0].to(device)     
        labels = seeds_label[1].to(device)
        optimizer.zero_grad()
        
        loss = 0
        train_accuracy = 0
        for i, seeds_i in enumerate(seeds):
            

            
            seeds_i = encoder(seeds_i).detach()
            seeds_i = seeds_i.expand(seeds.shape[1], -1)
            
            y_i = labels[i]
            y_hat = forward_model(seeds_i, encode_inflected, edge_index)
            
            
            
            _, top_indices_true = torch.topk(y_i.clone(), top_num)
            label_2 = torch.zeros(y_i.shape).to(device)
            label_2[top_indices_true] = 1
            
            _, top_indices_predict = torch.topk(y_hat.clone().squeeze(-1), top_num)
            
            # 将张量数组转换为Python列表
            list1 = top_indices_true.tolist()
            list_pre = top_indices_predict.tolist()

            # 使用集合操作找到交集
            intersection = list(set(list1) & set(list_pre))
            accuracy_i = len(intersection) / top_num       
            train_accuracy += accuracy_i 

            forward_loss = 0.5*F.mse_loss(y_hat.squeeze(-1), y_i, reduction='sum') + F.mse_loss(y_hat.squeeze(-1), label_2, reduction='sum')    
            loss += forward_loss    
        
           
        train_accuracy /= seeds.size(0)
        total_train_accuracy += train_accuracy
        loss = loss/seeds.size(0)
        total_loss += loss.item() 
        loss.backward()
        optimizer.step()
        # for p in forward_model.parameters():
        #     p.data.clamp_(min=0)
        

    print("Epoch: {}".format(epoch+1), 
        "\tTotal: {:.4f}".format(total_loss / count_train),
        "\tMean_train_accuracy: {:.4f}".format(total_train_accuracy/ count_train),
        )  
    
    total_test_accuracy = 0
    
    count_test = 0

    for batch_idx, seeds_label in enumerate(test_loader): 
        count_test += 1 
        seeds =  seeds_label[0].to(device)     
        labels = seeds_label[1].to(device)
        test_accuracy = 0
        for i, seeds_i in enumerate(seeds):
            seeds_i = encoder(seeds_i).detach()
            seeds_i = seeds_i.expand(seeds.shape[1], -1)
            
            y_i = labels[i]
            
            y_hat = forward_model(seeds_i, encode_inflected, edge_index)
            
            _, top_indices_true = torch.topk(y_i, top_num)
            
            
            _, top_indices_predict = torch.topk(y_hat.squeeze(-1), top_num)

            
            # 将张量数组转换为Python列表
            list1 = top_indices_true.tolist()
            list_pre = top_indices_predict.tolist()
            

            # 使用集合操作找到交集
            intersection = list(set(list1) & set(list_pre))

            
            accuracy_i = len(intersection) / top_num       
            test_accuracy += accuracy_i 
        test_accuracy /= len(seeds)
        total_test_accuracy += test_accuracy
        

    print(
        "\tMean_test_accuracy: {:.4f}".format(total_test_accuracy / count_test),
        )  

    

Epoch: 1 	Total: 355.9025 	Mean_train_accuracy: 0.5877
	Mean_test_accuracy: 0.5870
Epoch: 2 	Total: 355.2913 	Mean_train_accuracy: 0.5883
	Mean_test_accuracy: 0.5873
Epoch: 3 	Total: 355.2350 	Mean_train_accuracy: 0.5887
	Mean_test_accuracy: 0.5885
Epoch: 4 	Total: 355.6880 	Mean_train_accuracy: 0.5873
	Mean_test_accuracy: 0.5878
Epoch: 5 	Total: 355.7954 	Mean_train_accuracy: 0.5882
	Mean_test_accuracy: 0.5876
Epoch: 6 	Total: 355.6053 	Mean_train_accuracy: 0.5882
	Mean_test_accuracy: 0.5873
Epoch: 7 	Total: 355.2493 	Mean_train_accuracy: 0.5881
	Mean_test_accuracy: 0.5845
Epoch: 8 	Total: 355.5707 	Mean_train_accuracy: 0.5880
	Mean_test_accuracy: 0.5889
Epoch: 9 	Total: 355.4074 	Mean_train_accuracy: 0.5882
	Mean_test_accuracy: 0.5857
Epoch: 10 	Total: 355.5667 	Mean_train_accuracy: 0.5883
	Mean_test_accuracy: 0.5864
Epoch: 11 	Total: 355.2705 	Mean_train_accuracy: 0.5886
	Mean_test_accuracy: 0.5872
Epoch: 12 	Total: 355.9001 	Mean_train_accuracy: 0.5878
	Mean_test_accuracy: 0.5845
E